In [2]:
import pandas as pd

# Load datasets
global_monthly = pd.read_csv("../data/raw/Globalmonthlyandseasonal.csv")
southern_hemisphere = pd.read_csv("../data/raw/SouthernHemisphere.csv")
zone_annual = pd.read_csv("../data/raw/ZoneAnnual.csv")

# Inspect data
print(global_monthly.tail())
print(southern_hemisphere.tail())
print(zone_annual.tail())

# Cleaning - handle missing values, renaming columns, type conversion, etc.
global_monthly.replace("***", pd.NA, inplace=True)
southern_hemisphere.replace("***", pd.NA, inplace=True)

# Save cleaned data
global_monthly.to_csv("../data/cleaned/cleaned_globalmonthlyandseasonal.csv", index=False)
southern_hemisphere.to_csv("../data/cleaned/cleaned_southernhemisphere.csv", index=False)
zone_annual.to_csv("../data/cleaned/cleaned_zoneannual.csv", index=False)


                                                                                          Land-Ocean: Global Means
2020 1.17 1.24 1.17 1.12 1.01 .91  .89  .86  .98  .87  1.10 .79  1.01 1.04 1.17 1.10 .89                       .98
2021 .81  .64  .89  .76  .79  .84  .92  .82  .92  .99  .93  .87  .85  .84  .75  .81  .86                       .95
2022 .91  .89  1.05 .84  .84  .92  .94  .95  .89  .96  .73  .80  .89  .90  .89  .91  .94                       .86
2023 .87  .96  1.22 .99  .94  1.08 1.19 1.19 1.48 1.34 1.42 1.35 1.17 1.12 .88  1.05 1.15                     1.41
2024 1.24 1.43 1.40 1.32 1.16 1.25 1.21 1.30 ***  ***  ***  ***  ***  ***  1.34 1.30 1.25                      ***
                                                                             Land-Ocean: Southern Hemispheric Means
2020 .75 .77 .67  .85 .75 .71 .69 .62  .76  .55 .60 .39 .67 .71 .77 .76 .67                                     .63
2021 .34 .32 .56  .38 .53 .49 .77 .62  .79  .68 .57 .56 .55 .54 .35 .49 .63   